In [1]:
#imports
import pandas as pd
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from fpdf import FPDF
import matplotlib.pyplot as plt
import getpass

In [3]:
# functions
def rem_blanks(s):
    if s == None:
        return ''
    elif s[0] == ' ':
        return rem_blanks(s[1:])
    else:
        return s

In [4]:
def main():
    
    while True:
        print(
            '''
            Income Statement Analysis
            --------------------------

            Step 1:
            Visit www.morningstar.com, and search your desired company in the search bar.

            Step 2:
            Within the company's profile, click the "financials" tab.

            Step 3.
            With "Income Statement" displayed, click the "Details View" button.

            Step 4. 
            Leave all settings as they are, and click "Export to Excel," and save it to your Desktop.
            
            Step 5.
            Check to make sure the file saved in .xlsx format, otherwise manually save it as .xlsx.

            Step 6.
            Once step 1 through 5 are complete, enter your company's name.

            Step 7.
            Enter the excel sheet's file path (ex. /Users/username/Desktop/excelsheet.xlsx).

            '''
        )
        break

    # CLEAN & SAVE
    company = input('Enter company name: ')
    file_path = input('Enter file path: ')
    wb = load_workbook(file_path)
    ws = wb.active
    ws['A1'] = 'Categories'
    for row in range(1, 66):
        for col in range(1, 2):
            char = get_column_letter(col)
            ws[char + str(row)].value = rem_blanks(ws[char + str(row)].value)
    wb.save('/Users/' + getpass.getuser() + '/Desktop/' + f'{company}_financials.xlsx')

    # FINANCIAL RATIOS
    new_file_path = '/Users/' + getpass.getuser() + '/Desktop/' + f'{company}_financials.xlsx'

    df = pd.read_excel(new_file_path)
    gp = df.index[df['Categories'] == 'Gross Profit'].tolist()
    gross_profit = df['2020'].iloc[gp[0]]
    tr = df.index[df['Categories'] == 'Total Revenue'].tolist()
    total_revenue = df['2020'].iloc[tr[0]]
    pi = df.index[df['Categories'] == 'Pretax Income'].tolist()
    pretax_income = df['2020'].iloc[pi[0]]
    op = df.index[df['Categories'] == 'Net Income from Continuing Operations'].tolist()
    operating_profit = df['2020'].iloc[op[0]]
    eps = df.index[df['Categories'] == 'Basic EPS'].tolist()
    earnings_per_share = df['2020'].iloc[eps[0]]
    deps = df.index[df['Categories'] == 'Diluted EPS'].tolist()
    diluted_eps = df['2020'].iloc[deps[0]]

    def gross_margin(profit, revenue):
        return "{:.2%}".format(int(profit.replace(',', ''))/int(revenue.replace(',', '')))

    def profit_margin(pretax, revenue):
        return "{:.2%}".format(int(pretax.replace(',', ''))/int(revenue.replace(',', '')))

    def operating_margin(operating, revenue):
        return "{:.2%}".format(int(operating.replace(',', ''))/int(revenue.replace(',', '')))

    def basic_earnings_per_share(eps):
        return f'{eps}%'

    def diluted_earnings_per_share(deps):
        return f'{deps}%'

    # CREATE GRAPH
    tr = df.index[df['Categories'] == 'Total Revenue'].tolist()
    gp = df.index[df['Categories'] == 'Gross Profit'].tolist()

    tr_2016 = int(df['2016'].iloc[tr[0]].replace(',', ''))
    tr_2017 = int(df['2017'].iloc[tr[0]].replace(',', ''))
    tr_2018 = int(df['2018'].iloc[tr[0]].replace(',', ''))
    tr_2019 = int(df['2019'].iloc[tr[0]].replace(',', ''))
    tr_2020 = int(df['2020'].iloc[tr[0]].replace(',', ''))

    gp_2016 = int(df['2016'].iloc[gp[0]].replace(',', ''))
    gp_2017 = int(df['2017'].iloc[gp[0]].replace(',', ''))
    gp_2018 = int(df['2018'].iloc[gp[0]].replace(',', ''))
    gp_2019 = int(df['2019'].iloc[gp[0]].replace(',', ''))
    gp_2020 = int(df['2020'].iloc[gp[0]].replace(',', ''))

    x = [2016, 2017, 2018, 2019, 2020]

    y = [tr_2016/1000000000, tr_2017/1000000000, tr_2018/1000000000, tr_2019/1000000000, tr_2020/1000000000]
    y2 = [gp_2016/1000000000, gp_2017/1000000000, gp_2018/1000000000, gp_2019/1000000000, gp_2020/1000000000]

    plt.plot(x, y, linewidth=2, marker = '.', markersize = 10, label = 'Total Revenue')

    plt.plot(x, y2, linewidth=2, marker = '.', markersize = 10, label = 'Gross Profit')

    plt.title('Revenue vs Profit by Year', fontdict = {'fontname': 'Arial', 'fontsize': 16})
    plt.xlabel('Years', fontdict = {'fontname': 'Arial', 'fontsize': 12})
    plt.ylabel('Dollars (billions)', fontdict = {'fontname': 'Arial', 'fontsize': 12})
    plt.xticks([2016, 2017, 2018, 2019, 2020])
    plt.grid()
    plt.legend(loc = 'best', fontsize = 12)

    plt.savefig('/Users/' + getpass.getuser() + '/Desktop/' + 'figure1.png')

    # GENERATE PDF
    WIDTH = 210
    HEIGHT = 297

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Arial', 'B', 20)
    pdf.cell(w = 0, h = 10, txt = f'{company} Income Statement Analysis', border = 0, ln = 0, align = 'C')
    pdf.ln(15)
    pdf.image('figure1.png', 35, 30, WIDTH/1.5)

    pdf.ln(100)
    
    pdf.set_font('Arial', '', 16)
    pdf.cell(w = 0, h = 0, txt = f'Gross Margin: {gross_margin(gross_profit, total_revenue)}', border = 0, ln = 2)
    pdf.cell(w = 0, h = 15, txt = f'Profit Margin: {profit_margin(pretax_income, total_revenue)}', border = 0, ln = 1)
    pdf.cell(w = 0, h = 0, txt = f'Operating Margin: {operating_margin(operating_profit, total_revenue)}', border = 0, ln = 2)
    pdf.cell(w = 0, h = 15, txt = f'Basic EPS: {basic_earnings_per_share(earnings_per_share)}', border = 0, ln = 1)
    pdf.cell(w = 0, h = 0, txt = f'Diluted EPS: {diluted_earnings_per_share(diluted_eps)}', border = 0, ln = 2)

    pdf.output('/Users/' + getpass.getuser() + '/Desktop/' + f'{company}_report.pdf')

In [ ]:
# RUN
main()